In [ ]:
#make sure mongodb is running

In [ ]:
import os
from pymongo import MongoClient
import shutil
import utils.const as const

In [ ]:
db=const.grpId
collection=const.PostCollection
directory='attachment'
output=db

In [ ]:
if not os.path.isdir(directory):
  raise Exception('first download attachments ')
if os.path.isdir(output):
  raise Exception('delete '+output)
os.mkdir(output)

In [ ]:
client = MongoClient()
client.list_database_names() # ['admin', 'local']

In [ ]:
client = MongoClient()
myCollect=client[db][collection]

In [ ]:
# The code in the following cell is using shutil.copy().... if in limited memory or large data prefer using shutil.move()

In [ ]:
count=0;

for post in list(myCollect.find({})):
  if '_id' in post:
    A=os.path.join(output,post['_id'])
    os.mkdir(A)
    
    if 'attachments' in post:
      B=os.path.join(A,'attachments')
      os.mkdir(B)

      if 'videos' in post['attachments']:
        C=os.path.join(B,'videos')
        os.mkdir(C)

        myCollect.update_one({'_id': post['_id']}, {
                          '$unset': {'attachments.videos': ""}})
        
        for i in range(len(post['attachments']['videos'])):
          D=os.path.join(C,str(i))

          myCollect.update_one({'_id': post['_id']}, {
                              '$push': {'attachments.videos': i}})
          
          shutil.copy(os.path.join(directory,post['attachments']['videos'][i]),D)

          count+=1
      

      if 'images' in post['attachments']:
        C=os.path.join(B,'images')
        os.mkdir(C)

        myCollect.update_one({'_id': post['_id']}, {
                          '$unset': {'attachments.images': ""}})
        
        for i in range(len(post['attachments']['images'])):
          D=os.path.join(C,str(i))

          myCollect.update_one({'_id': post['_id']}, {
                              '$push': {'attachments.images': i}})
          
          shutil.copy(os.path.join(directory,post['attachments']['images'][i]),D)

          count+=1

          
      
          
    
    if 'comments' in post:
      B=os.path.join(A,'comments')
      os.mkdir(B)

      for comment in post['comments']:
        if '_id' in comment:
          C=os.path.join(B,comment['_id'])
          os.mkdir(C)

          if 'attachment' in comment:
            D=os.path.join(C,'attachment')
            os.mkdir(D)

            if 'videos' in comment['attachment']:
              E=os.path.join(D,'videos')
              os.mkdir(E)

              myCollect.update_one({'_id': post['_id'], 'comments': {'$elemMatch': {'_id': comment['_id']}}}, {
                  '$unset': {'comments.$.attachment.videos': ""}})

              for i in range(len(comment['attachment']['videos'])):
                F=os.path.join(E,str(i))

                myCollect.update_one({'_id': post['_id'], 'comments': {'$elemMatch': {'_id': comment['_id']}}}, {
                              '$push': {'comments.$.attachment.videos': i}})

                shutil.copy(os.path.join(directory,comment['attachment']['videos'][i]),F)
                count+=1

            if 'images' in comment['attachment']:
              E=os.path.join(D,'images')
              os.mkdir(E)

              myCollect.update_one({'_id': post['_id'], 'comments': {'$elemMatch': {'_id': comment['_id']}}}, {
                  '$unset': {'comments.$.attachment.images': ""}})

              for i in range(len(comment['attachment']['images'])):
                F=os.path.join(E,str(i))

                myCollect.update_one({'_id': post['_id'], 'comments': {'$elemMatch': {'_id': comment['_id']}}}, {
                              '$push': {'comments.$.attachment.images': i}})

                shutil.copy(os.path.join(directory,comment['attachment']['images'][i]),F)
                count+=1


In [ ]:
print(count)

In [ ]:
os.system('python3 dump.py')

In [ ]:
#have a look at json_dump.json